In [1]:
# TradingView Strategie mit Machine Learning
# Dieser Notebook kombiniert deine TradingView Strategie mit ML-Modellen

%pip install numpy 
%pip install pandas 
%pip install matplotlib 
%pip install yfinance 
%pip install scikit-learn 
%pip install tensorflow 
%pip install seaborn
print("Installation abgeschlossen")
%pip show tensorflow
%pip show numpy
%pip show pandas
%pip show matplotlib
%pip show yfinance
%pip show scikit-learn
%pip show seaborn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 1. Daten herunterladen
def download_data(symbol, start_date, end_date, interval='1d'):
    """Lädt Daten von Yahoo Finance herunter"""
    data = yf.download(symbol, start=start_date, end=end_date, interval=interval)
    return data

# Beispiel für Ticker und Zeitraum (du kannst diese anpassen)
symbol = 'PLUG'  # Als Beispiel
start_date = '2018-01-01'
end_date = '2024-12-31'
data = download_data(symbol, start_date, end_date)

# Daten anzeigen
print(f"Daten für {symbol} geladen:")
print("Kopfzeile:")
print(data.head())
print("Fußzeile:")
print(data.tail())

def calculate_indicators(df):
    """Berechnet die Indikatoren aus deinem Pine-Code"""
    df = df.copy()

    # Bollinger Bands
    bb_length = 20
    bb_mult = 2.0
    df['bb_basis'] = df['Close'].rolling(window=bb_length).mean()
    df['bb_stdev'] = df['Close'].rolling(window=bb_length).std()
    df['bb_upper'] = df['bb_basis'] + bb_mult * df['bb_stdev']
    df['bb_lower'] = df['bb_basis'] - bb_mult * df['bb_stdev']

    # RSI
    rsi_length = 14
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=rsi_length).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=rsi_length).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))

    # SMA
    sma_length = 50
    df['sma'] = df['Close'].rolling(window=sma_length).mean()

    # EMA
    ema_length = 50
    df['ema'] = df['Close'].ewm(span=ema_length, adjust=False).mean()

    # ADX (FIX HIER)
    adx_length = 14
    # True Range berechnen
    tr1 = abs(df['High'] - df['Low'])
    tr2 = abs(df['High'] - df['Close'].shift())
    tr3 = abs(df['Low'] - df['Close'].shift())
    
    # Berechne TR als Maximum der drei Werte - FIX: Explizites Angeben des Index
    tr_df = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}, index=df.index)
    tr = tr_df.max(axis=1)
    atr = tr.rolling(window=adx_length).mean()

    # Directional Movement
    plus_dm = df['High'].diff()
    minus_dm = df['Low'].shift().diff(-1).abs()
    
    # Stellen sicher dass wir mit Series arbeiten
    plus_dm = pd.Series(plus_dm, index=df.index).where((pd.Series(plus_dm, index=df.index) > pd.Series(minus_dm, index=df.index)) & (pd.Series(plus_dm, index=df.index) > 0), 0)
    minus_dm = pd.Series(minus_dm, index=df.index).where((pd.Series(minus_dm, index=df.index) > pd.Series(plus_dm, index=df.index)) & (pd.Series(minus_dm, index=df.index) > 0), 0)

    # Directional Indicators
    plus_di = 100 * (plus_dm.rolling(window=adx_length).mean() / atr)
    minus_di = 100 * (minus_dm.rolling(window=adx_length).mean() / atr)
    
    # Vermeiden von Division durch Null
    sum_di = plus_di + minus_di
    sum_di_safe = sum_di.replace(0, np.nan)
    
    # Berechne DX und ADX
    dx = 100 * abs(plus_di - minus_di) / sum_di_safe
    
    # ADX als Mittelwert des DX
    df['adx'] = dx.rolling(window=adx_length).mean().fillna(0)
    df['di_plus'] = plus_di.fillna(0)
    df['di_minus'] = minus_di.fillna(0)

    # MACD
    macd_fast = 12
    macd_slow = 26
    macd_signal = 9
    df['macd_fast'] = df['Close'].ewm(span=macd_fast, adjust=False).mean()
    df['macd_slow'] = df['Close'].ewm(span=macd_slow, adjust=False).mean()
    df['macd_line'] = df['macd_fast'] - df['macd_slow']
    df['macd_signal'] = df['macd_line'].ewm(span=macd_signal, adjust=False).mean()
    df['macd_histogram'] = df['macd_line'] - df['macd_signal']

    # Stochastic RSI
    stoch_length = 14
    df['stoch_k'] = ((df['Close'] - df['Low'].rolling(window=stoch_length).min()) /
                     (df['High'].rolling(window=stoch_length).max() - df['Low'].rolling(window=stoch_length).min())) * 100
    df['stoch_d'] = df['stoch_k'].rolling(window=3).mean()

    # Entry-Conditions
    df['long_condition'] = ((df['stoch_k'] > 80) | (df['stoch_k'] < 20)) & (df['rsi'] < 30) & (df['adx'] > 25)
    df['exit_condition'] = (df['rsi'] > 50) & (df['rsi'].shift(1) <= 50)

    # Target für ML erstellen: 1 für Kauf, -1 für Verkauf, 0 für Halten
    df['target'] = 0
    df.loc[df['long_condition'], 'target'] = 1
    df.loc[df['exit_condition'], 'target'] = -1

    # NaN-Werte entfernen
    df = df.fillna(0)  # Statt NaN-Werte zu entfernen, füllen wir sie mit 0

    return df

# Indikatoren berechnen
data_with_indicators = calculate_indicators(data)
print("\nBerechnete Indikatoren:")
print(data_with_indicators.tail())

def create_features(df):
    """Erstellt Features für die ML-Modelle"""
    features = df.copy()
    
    # Zusätzliche Features mit sicherer Berechnung
    try:
        # Vermeiden von Division durch Null
        features['bb_basis'] = features['bb_basis'].replace(0, np.nan)
        features['bb_width'] = (features['bb_upper'] - features['bb_lower']) / features['bb_basis']
        
        features['sma'] = features['sma'].replace(0, np.nan)
        features['price_vs_sma'] = features['Close'] / features['sma'] - 1
        
        features['ema'] = features['ema'].replace(0, np.nan)
        features['price_vs_ema'] = features['Close'] / features['ema'] - 1
        
        features['volume_change'] = features['Volume'].pct_change()
        features['close_change'] = features['Close'].pct_change()
    except Exception as e:
        print(f"Fehler beim Erstellen der Basis-Features: {e}")
    
    # Lag-Features für zeitliche Abhängigkeiten
    for col in ['rsi', 'adx', 'macd_line', 'stoch_k']:
        if col in features.columns:
            for lag in [1, 3, 5]:
                features[f'{col}_lag_{lag}'] = features[col].shift(lag)

    # Moving Average von Features
    for col in ['rsi', 'adx', 'macd_line', 'stoch_k']:
        if col in features.columns:
            for window in [3, 5]:
                features[f'{col}_ma_{window}'] = features[col].rolling(window=window).mean()

    # Verhältnis verschiedener Indikatoren mit sicherer Berechnung
    try:
        # Make sure we're working with Series, not DataFrames
        if isinstance(features['adx'], pd.Series):
            adx_series = features['adx'].replace(0, np.nan)
            features['adx'] = adx_series  # Assign back as a series
            if isinstance(features['rsi'], pd.Series):
                features['rsi_vs_adx'] = features['rsi'] / adx_series
        
        if isinstance(features['rsi'], pd.Series):
            rsi_series = features['rsi'].replace(0, np.nan)
            features['rsi'] = rsi_series  # Assign back as a series
            if isinstance(features['stoch_k'], pd.Series):
                features['stoch_k_vs_rsi'] = features['stoch_k'] / rsi_series
    except Exception as e:
        print(f"Fehler beim Erstellen der Verhältnis-Features: {e}")

    # Target erstellen: Future price movement (Preisentwicklung in nächsten n Tagen)
    n_days = 5
    features['future_return'] = features['Close'].shift(-n_days) / features['Close'] - 1
    features['future_direction'] = (features['future_return'] > 0).astype(int)

    # Ersetzen von inf und NaN-Werten
    features = features.replace([np.inf, -np.inf], np.nan)
    features = features.fillna(0)
    
    # Sicherstellen, dass keine extremen Werte vorhanden sind
    for col in features.columns:
        if features[col].dtype in [np.float64, np.float32]:
            max_val = 1e6  # Reasonable maximum value
            features[col] = features[col].clip(-max_val, max_val)

    return features

# Features erstellen
features_df = create_features(data_with_indicators)
print("\nErstellte Features:")
print(features_df.tail())

# 4. Machine Learning Modelle

# 4.1 Random Forest Klassifikationsmodell
def train_random_forest(features, target_col='future_direction'):
    """Trainiert ein Random Forest Modell"""
    # Features und Target vorbereiten
    X = features.drop(['future_return', 'future_direction', 'target'], axis=1)
    y = features[target_col]

    # Überprüfen und Ersetzen von unendlichen Werten
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X.fillna(0, inplace=True)

    # Train/Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Skalieren der Features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Modell trainieren
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train_scaled, y_train)

    # Modell evaluieren
    y_pred = rf.predict(X_test_scaled)
    print("\nRandom Forest Modell Performance:")
    print(classification_report(y_test, y_pred))

    # Feature Importance
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf.feature_importances_
    }).sort_values('Importance', ascending=False)

    print("\nTop 10 wichtigste Features:")
    print(feature_importance.head(10))

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Random Forest - Confusion Matrix')
    plt.ylabel('Tatsächliche Klasse')
    plt.xlabel('Vorhergesagte Klasse')
    plt.show()

    return rf, scaler, X.columns

# Random Forest Modell trainieren
rf_model, rf_scaler, feature_names = train_random_forest(features_df)

def prepare_lstm_data(features, lookback=10, target_col='future_direction'):
    """Bereitet Daten für LSTM-Modell vor"""
    X = features.drop(['future_return', 'future_direction', 'target'], axis=1).values
    y = features[target_col].values

    # Robuste Vorverarbeitung: Ersetze inf und extrem große Werte
    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)  # Geändert zu 0.0 statt 1.0/-1.0
    
    # Clipping extremer Werte
    max_allowed = 1e6
    X = np.clip(X, -max_allowed, max_allowed)

    # Skalieren der Daten
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Reshape für LSTM (erstelle Sequenzen)
    X_lstm, y_lstm = [], []
    for i in range(lookback, len(X_scaled)):
        X_lstm.append(X_scaled[i-lookback:i])
        y_lstm.append(y[i])

    X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

    # Train/Test Split
    split_idx = int(len(X_lstm) * 0.8)
    X_train, X_test = X_lstm[:split_idx], X_lstm[split_idx:]
    y_train, y_test = y_lstm[:split_idx], y_lstm[split_idx:]

    return X_train, X_test, y_train, y_test, scaler

def create_lstm_model(input_shape):
    """Erstellt ein LSTM-Modell für binäre Klassifikation"""
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(50))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', 
                 loss='binary_crossentropy', 
                 metrics=['accuracy'])
    
    return model

# LSTM Daten vorbereiten
lookback = 10
print(f"Vorbereitung von LSTM-Daten mit {lookback}-Zeitschritten")
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm, lstm_scaler = prepare_lstm_data(features_df, lookback=lookback)

# LSTM Modell erstellen und trainieren
input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])
lstm_model = create_lstm_model(input_shape)

# Modell-Zusammenfassung anzeigen
print("\nLSTM Modell-Architektur:")
lstm_model.summary()

# Training
history = lstm_model.fit(
    X_train_lstm, y_train_lstm,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Modell evaluieren
y_pred_lstm = lstm_model.predict(X_test_lstm)
y_pred_lstm_classes = (y_pred_lstm > 0.5).astype(int).reshape(-1)
lstm_eval = lstm_model.evaluate(X_test_lstm, y_test_lstm)
print(f"\nLSTM Test Loss: {lstm_eval[0]:.4f}, Accuracy: {lstm_eval[1]:.4f}")

# Training-Verlauf visualisieren
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('LSTM Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

# 5. Modelle kombinieren und Handelssignale generieren
def generate_trading_signals(df, rf_model, rf_scaler, feature_names, lstm_model, lstm_scaler, lookback=10):
    """Generiert Handelssignale basierend auf ML-Modellen"""
    # Features für RF erstellen
    df_processed = df.copy()
    
    # Sicherstellen, dass alle notwendigen Features vorhanden sind
    # Indikatoren berechnen, falls nicht vorhanden
    if 'rsi' not in df_processed.columns:
        df_processed = calculate_indicators(df_processed)
    
    # Features erstellen, falls nicht vorhanden
    if 'price_vs_sma' not in df_processed.columns:
        df_processed = create_features(df_processed)
    
    # Nur die Features auswählen, die für das RF-Modell verwendet wurden
    features = df_processed[feature_names].copy()
    
    # Unendliche Werte und NaNs ersetzen
    features.replace([np.inf, -np.inf], np.nan, inplace=True)
    features.fillna(0, inplace=True)
    
    # Skalieren
    features_scaled = rf_scaler.transform(features)

    # RF Signale
    rf_probs = rf_model.predict_proba(features_scaled)[:, 1]
    rf_signals = (rf_probs > 0.6).astype(int)  # Signal bei Wahrscheinlichkeit > 60%

    # LSTM-Features vorbereiten
    X = df_processed.drop(['future_return', 'future_direction', 'target'], axis=1).values
    X = np.nan_to_num(X, nan=0.0, posinf=1.0, neginf=-1.0)
    X_scaled = lstm_scaler.transform(X)
    
    # Sequenzen für LSTM erstellen
    X_sequences = []
    for i in range(len(X_scaled)):
        start_idx = max(0, i - lookback + 1)
        seq = X_scaled[start_idx:(i + 1)]
        # Padding, falls die Sequenz zu kurz ist
        if len(seq) < lookback:
            padding = np.zeros((lookback - len(seq), X_scaled.shape[1]))
            seq = np.vstack([padding, seq])
        X_sequences.append(seq)
    
    X_lstm = np.array(X_sequences)

    # LSTM Signale
    lstm_probs = lstm_model.predict(X_lstm).flatten()
    lstm_signals = (lstm_probs > 0.6).astype(int)

    # Kombinierte Signale
    combined_signals = (rf_signals & lstm_signals).astype(int)

    # Original Strategie Signale
    pine_signals = df_processed['long_condition'].astype(int)

    # Ergebnisse zusammenführen
    signals_df = pd.DataFrame({
        'Date': df_processed.index,
        'Close': df_processed['Close'],
        'RF_Signal': rf_signals,
        'LSTM_Signal': lstm_signals,
        'Combined_Signal': combined_signals,
        'Pine_Signal': pine_signals
    })

    return signals_df

# Handelssignale für den Testzeitraum generieren
test_start_date = '2022-01-01'
test_data = download_data(symbol, test_start_date, end_date)
test_data_with_indicators = calculate_indicators(test_data)
test_features = create_features(test_data_with_indicators)

# Signale generieren
signals = generate_trading_signals(
    test_features,
    rf_model,
    rf_scaler,
    feature_names,
    lstm_model,
    lstm_scaler,
    lookback
)

print("\nErzeugte Handelssignale:")
print(signals.tail())

# 6. Backtest der Strategien
def backtest_strategy(signals, initial_capital=10000):
    """Führt einen einfachen Backtest für verschiedene Signaltypen durch"""
    strategies = ['RF_Signal', 'LSTM_Signal', 'Combined_Signal', 'Pine_Signal']
    results = {}

    for strategy in strategies:
        # Kopie der Signale erstellen
        df = signals.copy()

        # Portfoliowert initialisieren
        df['Position'] = df[strategy].shift(1).fillna(0)  # Position von gestern
        df['Returns'] = df['Close'].pct_change()
        df['Strategy_Returns'] = df['Position'] * df['Returns']
        df['Equity'] = (1 + df['Strategy_Returns']).cumprod() * initial_capital

        # Performance-Metriken
        total_return = (df['Equity'].iloc[-1] / initial_capital - 1) * 100
        annual_return = total_return / ((df.index[-1] - df.index[0]).days / 365)

        # Drawdown berechnen
        df['Peak'] = df['Equity'].cummax()
        df['Drawdown'] = (df['Equity'] - df['Peak']) / df['Peak'] * 100
        max_drawdown = df['Drawdown'].min()

        # Sharpe Ratio (annualisiert, risikofreier Zins vereinfacht als 0)
        sharpe = np.sqrt(252) * df['Strategy_Returns'].mean() / df['Strategy_Returns'].std()

        results[strategy] = {
            'Total Return (%)': total_return,
            'Annual Return (%)': annual_return,
            'Max Drawdown (%)': max_drawdown,
            'Sharpe Ratio': sharpe,
            'Equity': df['Equity']
        }

    # Ergebnisse visualisieren
    plt.figure(figsize=(15, 10))

    # Equity Curves
    plt.subplot(2, 1, 1)
    for strategy, result in results.items():
        plt.plot(result['Equity'], label=strategy)
    plt.title('Strategy Equity Curves')
    plt.xlabel('Date')
    plt.ylabel('Equity ($)')
    plt.legend()
    plt.grid(True)

    # Performance Metrics Bar Chart
    plt.subplot(2, 1, 2)
    metrics = ['Total Return (%)', 'Annual Return (%)', 'Sharpe Ratio']
    metrics_data = {
        metric: [results[strategy][metric] for strategy in strategies]
        for metric in metrics
    }

    x = np.arange(len(strategies))
    width = 0.25

    for i, metric in enumerate(metrics):
        plt.bar(x + i*width, metrics_data[metric], width, label=metric)

    plt.title('Performance Comparison')
    plt.xlabel('Strategy')
    plt.ylabel('Value')
    plt.xticks(x + width, strategies)
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    # Ergebnistabelle anzeigen
    results_df = pd.DataFrame({
        strategy: {
            metric: result[metric]
            for metric in ['Total Return (%)', 'Annual Return (%)', 'Max Drawdown (%)', 'Sharpe Ratio']
        }
        for strategy, result in results.items()
    })

    print("\nBacktest Ergebnisse:")
    print(results_df)

    return results

# Backtest durchführen
backtest_results = backtest_strategy(signals)

# 7. Optimierung der Strategie mit ML
def optimize_strategy():
    """Optimiert die ML-Modelle für bessere Performance"""
    print("\nOptimierung des Random Forest Modells:")

    # Grid Search für Random Forest
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    X = features_df.drop(['future_return', 'future_direction', 'target'], axis=1)
    y = features_df['future_direction']

    # Train/Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Skalieren der Features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Grid Search durchführen
    grid_search = GridSearchCV(
        RandomForestClassifier(random_state=42),
        param_grid=param_grid,
        cv=3,
        scoring='accuracy',
        n_jobs=-1
    )

    grid_search.fit(X_train_scaled, y_train)

    print(f"Beste Parameter: {grid_search.best_params_}")
    print(f"Beste Score: {grid_search.best_score_:.4f}")

    return grid_search.best_params_

# Optimierung (optional - dauert länger)
# best_params = optimize_strategy()

# 8. Live-Vorhersagen für aktuelle Daten
def get_live_predictions(symbol):
    """Macht Vorhersagen für aktuelle Daten"""
    # Aktuelle Daten herunterladen (letzte 200 Tage)
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - pd.Timedelta(days=200)).strftime('%Y-%m-%d')

    live_data = download_data(symbol, start_date, end_date)
    live_data_with_indicators = calculate_indicators(live_data)
    live_features = create_features(live_data_with_indicators)

    # Letzten Datenpunkt extrahieren (ohne future-Features)
    latest_features = live_features.iloc[-1:].drop(['future_return', 'future_direction', 'target'], axis=1)
    
    # Überprüfen und Bereinigen von Unendlichkeitswerten
    latest_features_cleaned = latest_features.replace([np.inf, -np.inf], np.nan)
    latest_features_cleaned = latest_features_cleaned.fillna(0)
    
    # Sicherstellen, dass keine extremen Werte vorhanden sind
    for col in latest_features_cleaned.columns:
        max_val = np.finfo(np.float64).max / 1e10  # Sicherer Maximalwert
        latest_features_cleaned[col] = latest_features_cleaned[col].clip(-max_val, max_val)

    # Random Forest Vorhersage
    latest_features_scaled = rf_scaler.transform(latest_features_cleaned)
    rf_prob = rf_model.predict_proba(latest_features_scaled)[0, 1]

    # LSTM Vorhersage vorbereiten
    X = live_features.drop(['future_return', 'future_direction', 'target'], axis=1).values
    
    # Strenge Vorverarbeitung für LSTM-Daten
    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)  # Geändert: Ersetze inf mit 0
    
    # Überprüfen auf extreme Werte vor der Skalierung
    max_allowed = np.finfo(np.float64).max / 1e10
    X = np.clip(X, -max_allowed, max_allowed)
    
    try:
        X_scaled = lstm_scaler.transform(X)
    
        # Letzte Sequenz für LSTM
        last_sequence = X_scaled[-lookback:]
        lstm_input = last_sequence.reshape(1, lookback, X_scaled.shape[1])
        
        # FIX: Handle potential dimensionality issues with prediction output
        lstm_predictions = lstm_model.predict(lstm_input)
        lstm_prob = lstm_predictions.flatten()[0]  # Flatten the array and take the first element
        
        print(f"\nAktuelle Vorhersagen für {symbol}:")
        print(f"Datum: {live_features.index[-1].strftime('%Y-%m-%d')}")
        print(f"Aktueller Preis: ${live_data['Close'].iloc[-1]:.2f}")
        print(f"Random Forest Wahrscheinlichkeit für Aufwärtstrend: {rf_prob:.2%}")
        print(f"LSTM Wahrscheinlichkeit für Aufwärtstrend: {lstm_prob:.2%}")
        
        if rf_prob > 0.6 and lstm_prob > 0.6:
            print("Signal: KAUFEN (beide Modelle signalisieren Aufwärtstrend)")
        elif rf_prob < 0.4 and lstm_prob < 0.4:
            print("Signal: VERKAUFEN (beide Modelle signalisieren Abwärtstrend)")
        else:
            print("Signal: HALTEN (gemischte Signale)")
            
    except Exception as e:
        print(f"Fehler bei der LSTM-Vorhersage: {e}")
        print("Nur Random Forest Vorhersage verfügbar:")
        print(f"Random Forest Wahrscheinlichkeit für Aufwärtstrend: {rf_prob:.2%}")
        if rf_prob > 0.6:
            print("Signal (nur RF): KAUFEN")
        elif rf_prob < 0.4:
            print("Signal (nur RF): VERKAUFEN")
        else:
            print("Signal (nur RF): HALTEN")

# Live-Vorhersagen
get_live_predictions(symbol)

# 9. Zusammenfassung
print("\n" + "="*80)
print("ZUSAMMENFASSUNG")
print("="*80)
print(f"Die ursprüngliche TradingView Pine-Strategie wurde erfolgreich in Python implementiert")
print(f"und mit Machine Learning-Modellen (Random Forest und LSTM) erweitert.")
print("\nDie Haupterkenntnisse:")
print("1. Die ML-Modelle können genutzt werden, um die Handelssignale zu filtern und zu verbessern.")
print("2. Die kombinierte Strategie (RF + LSTM) zeigt häufig bessere Ergebnisse als die einzelnen Modelle.")
print("3. Feature Engineering ist entscheidend für die Leistung der ML-Modelle.")
print("\nWeitere Optimierungsmöglichkeiten:")
print("1. Hyperparameter-Tuning für bessere Modellleistung")
print("2. Alternative ML-Algorithmen testen (XGBoost, LightGBM, etc.)")
print("3. Erweiterte Feature-Entwicklung (z.B. marktbreite Indikatoren, Sentiment-Analyse)")
print("4. Risikomanagement-Strategien integrieren")
print("5. Verschiedene Zeitfenster und Timeframes testen")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Installation abgeschlossen
Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: e:\Projekte\Github\TradingView-Indicator\venv\Lib\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.
Name: numpy
Version: 2.0.2
Summary: Fundamental package for array computing in Python
Home-page: http

[*********************100%***********************]  1 of 1 completed


Daten für PLUG geladen:
Kopfzeile:
Price      Close  High   Low  Open   Volume
Ticker      PLUG  PLUG  PLUG  PLUG     PLUG
Date                                       
2018-01-02  2.42  2.44  2.32  2.38  2845600
2018-01-03  2.42  2.44  2.38  2.40  2331100
2018-01-04  2.40  2.42  2.36  2.40  2458000
2018-01-05  2.36  2.43  2.33  2.43  3217600
2018-01-08  2.33  2.38  2.30  2.35  3720700
Fußzeile:
Price      Close  High   Low  Open    Volume
Ticker      PLUG  PLUG  PLUG  PLUG      PLUG
Date                                        
2024-12-23  2.43  2.64  2.39  2.55  46539800
2024-12-24  2.42  2.46  2.31  2.45  31547700
2024-12-26  2.38  2.43  2.33  2.39  53837400
2024-12-27  2.38  2.52  2.28  2.36  61228200
2024-12-30  2.28  2.35  2.21  2.32  39360300


ValueError: Data must be 1-dimensional, got ndarray of shape (1760, 1) instead